In [1]:
import io
import re
import numpy as np
import pandas as pd
import numpy.linalg as lin
import warnings 
import pickle
from sklearn.cluster import KMeans
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate
warnings.filterwarnings('ignore')

In [2]:
stopwords=['ଦେଇଛନ୍ତି', 'ଲେଖାଏଁ', 'ଜଣେ', 'ଏହା', 'ତେଣୁ', 'ମିଳିଥାଏ', 'ପାଇଁ', 'ନେଉଛନ୍ତି', 'ଯୋଗୁଁ', 'ଏପର୍ଯ୍ୟନ୍ତ', 'ଏଭଳି', 'କରୁଛି', 'ଓ', 'ଯାଏଁ','ହୋଇଛନ୍ତି', 'କି', 'କରାଯିବା', 'ପରେ', 'ଏହି', 'ଏବଂ', 'ଜଣ', 'ଥିବା','ହୋଇଥିଲା', 'ତେବେ', 'ଆଜି', 'ଜଣଙ୍କ', 'ଏଥି', 'ଗତ', 'ହୋଇଥିଲେ',
       'ଦିନ ଧରି', 'ହେଉଥିବା', 'ଦିନ ତଳେ', 'ଯୋଗେ', 'ବୋଲି', 'ଜଣାପଡ଼ିଛି',
       'ଦ୍ବାରା', 'କରି', 'ଯାଇ', 'ଏ ନେଇ', 'ଚାଲୁ', 'ରହିଛି', 'ତାରିଖ',
       'ମିଳିଛି', 'ବର୍ଷୀୟ', 'ଦିନ ତଳେ', 'ସହ', 'ଆସିଛନ୍ତି', 'ମଧ୍ୟ', 'କେଉଁ',
       'ହୋଇଯାଇଛି', 'ନେଇ ଯାଇଛି', 'କାମ', 'କରିଛି', 'ହେବା', 'ପରେ', 'ଏବେ',
       'ହୋଇଛି', 'ତେବେ', 'ରଖି', 'ନିଆଯାଇଥିବା', 'ପର୍ଯ୍ୟନ୍ତ', 'କରିଛନ୍ତି',
       'ଉପରେ', 'ଦେଲେ', 'ଥର', 'ଏବଂ', 'ଆଉ']
alphabets=['ଅ','ଆ','ଇ','ଈ','ଉ','ଊ','ଋ','ଌ','ଏ','ଐ','ଓ','ଔ','କ','ଖ','ଗ','ଘ','ଙ','ଚ','ଛ','ଜ','ଝ','ଞ','ଟ','ଠ','ଡ','ଢ','ଣ',
 'ତ','ଥ','ଦ','ଧ','ନ','ପ','ଫ','ବ','ଭ','ମ','ଯ','ର','ଲ','ଳ','ଵ','ଶ','ଷ','ସ','ହ','ଡ଼','ଢ଼','ୟ','ୠ','ୡ','ୱ']
matra=['ଁ','ଂ','ଃ','଼','ା','ି','ୀ','ୁ','ୂ','ୃ','ୄ','େ','ୈ','ୋ','ୌ','୍','ୢ','ୣ']

In [3]:
delimiters=["।","?","!"]

In [4]:
file=open('sentence.pkl','rb')
verse_embeddings_sentence=pickle.load(file)
file.close()
file=open('max.pkl','rb')
verse_embeddings_max=pickle.load(file)
file.close()
file=open('mean.pkl','rb')
verse_embeddings_mean=pickle.load(file)
file.close()

In [5]:
file=open('whole.pkl','rb')
verse_embeddings_whole=pickle.load(file)
file.close()
file=open('glove.pkl','rb')
glove=pickle.load(file)
file.close()

In [6]:
def o2h(text):
    return transliterate(text, sanscript.ORIYA, sanscript.DEVANAGARI)

In [7]:
def remove_special_characters(text):
    string=''
    for i in text:
        if i in alphabets+matra+delimiters:
            string=string+i
        else:
            string=string+" "
    return " ".join(string.split())
def sent_tokenize(text):
    text=remove_special_characters(text)
    regexPattern = '|'.join(map(re.escape, delimiters))
    sentences=re.split(regexPattern,text)
    return [sent.strip() for sent in sentences if len(sent.strip())>0]

In [8]:
def word_embedding(word):
    try:
        return glove[o2h(word)]
    except:
        return np.array([0.0]*100)

In [9]:
def cosine_similarity(a,b):
    try:
        return a.dot(b)/(lin.norm(a)*lin.norm(b))
    except:
        return -1.0

In [10]:
def ed(a,b):
    return lin.norm(a-b)

In [11]:
def sentence_embedding(sentence):
    sentence=remove_special_characters(sentence)
    words=sentence.split()
    words=[word  for word in words if word not in stopwords]
    if len(words)>0:
        sentence_embedding=[word_embedding(word) for word in words]
        return np.array(list(map(lambda x: sum(x)/len(x), zip(*sentence_embedding))))
    return np.array([0.0]*100)

In [12]:
data=pd.read_csv('../odia.csv')

In [13]:
query='ସବୁଠାରୁ ଉପଯୁକ୍ତ କାର୍ଯ୍ୟ ପଦ୍ଧତି ବିଷୟରେ ଅନୁମାନ କରିବାକୁ ଯାଇ'
query_embedding=sentence_embedding(query)
query_embedding_list=query_embedding.reshape(1,-1).astype(np.float64)

# Max Strategy

## Cosine Similarity

In [14]:
def max_cosine(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_max]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [15]:
verse_index=max_cosine(query_embedding)

In [16]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
625,18,11,न हि देहभृता शक्यं त्यक्तुं कर्माण्यशेषतः।यस्त...,ଏଠାରେ ଯୁକ୍ତି କରାଯାଇପାରେ ଯେ କର୍ମଫଳ ତ୍ୟାଗ କରିବା ...
522,14,7,रजो रागात्मकं विद्धि तृष्णासङ्गसमुद्भवम्।तन्नि...,"ଶ୍ରୀକୃଷ୍ଣ ବର୍ତ୍ତମାନ ରଜୋଗୁଣର କାର୍ଯ୍ୟ, ଏବଂ ଏହା କ..."
193,4,39,श्रद्धावाँल्लभते ज्ञानं तत्परः संयतेन्द्रियः।\...,"ଶ୍ରୀକୃଷ୍ଣ ବର୍ତ୍ତମାନ, ଜ୍ଞାନର ପରିପ୍ରେକ୍ଷୀରେ, ଶ୍ର..."
44,2,5,गुरूनहत्वा हि महानुभावान्\n\nश्रेयो भोक्तुं भै...,ଏପରି ଯୁକ୍ତି କରାଯାଇ ପାରେ ଯେ ଅର୍ଜୁନ ନିଜର ଜୀବନ ନି...
197,5,1,अर्जुन उवाच\n\nसंन्यासं कर्मणां कृष्ण पुनर्योग...,ଅର୍ଜୁନଙ୍କର ଷୋହଳଟି ପ୍ରଶ୍ନ ମଧ୍ୟରୁ ଏହା ପଞ୍ଚମ ପ୍ରଶ...
43,2,4,अर्जुन उवाच\n\nकथं भीष्ममहं संख्ये द्रोणं च मध...,କର୍ମ କରିବା ପାଇଁ ଶ୍ରୀକୃଷ୍ଣଙ୍କ ଆହ୍ୱାନର ପ୍ରତିକ୍ରି...
219,5,23,शक्नोतीहैव यः सोढुं प्राक्शरीरविमोक्षणात्।\n\n...,ମନୁଷ୍ୟ ଶରୀର ଆତ୍ମାକୁ ତା’ର ଭଗବତ୍ ପ୍ରାପ୍ତି ରୂପୀ ଚ...
217,5,21,बाह्यस्पर्शेष्वसक्तात्मा विन्दत्यात्मनि यत्सुख...,ବୈଦିକ ଶାସ୍ତ୍ରସବୁ ଭଗବାନଙ୍କୁ ଅନନ୍ତ ଦିବ୍ୟ ଆନନ୍ଦର ...
201,5,5,यत्सांख्यैः प्राप्यते स्थानं तद्योगैरपि गम्यते...,"ଅଧ୍ୟାତ୍ମିକ ସାଧନାରେ, ବାହ୍ୟ କ୍ରିୟା ଅପେକ୍ଷା, ମନୋଭ..."
239,6,14,प्रशान्तात्मा विगतभीर्ब्रह्मचारिव्रते स्थितः।\...,ଧ୍ୟାନରେ ସଫଳତା ହାସଲ କରିବା ପାଇଁ ଶ୍ରୀକୃଷ୍ଣ ବ୍ରହ୍ମ...


## Euclidean Distance

In [17]:
def max_ed(query_embedding,counter=10):
    scoring=[ed(query_embedding,i) for i in verse_embeddings_max]
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [18]:
verse_index=max_ed(query_embedding)

In [19]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
625,18,11,न हि देहभृता शक्यं त्यक्तुं कर्माण्यशेषतः।यस्त...,ଏଠାରେ ଯୁକ୍ତି କରାଯାଇପାରେ ଯେ କର୍ମଫଳ ତ୍ୟାଗ କରିବା ...
193,4,39,श्रद्धावाँल्लभते ज्ञानं तत्परः संयतेन्द्रियः।\...,"ଶ୍ରୀକୃଷ୍ଣ ବର୍ତ୍ତମାନ, ଜ୍ଞାନର ପରିପ୍ରେକ୍ଷୀରେ, ଶ୍ର..."
44,2,5,गुरूनहत्वा हि महानुभावान्\n\nश्रेयो भोक्तुं भै...,ଏପରି ଯୁକ୍ତି କରାଯାଇ ପାରେ ଯେ ଅର୍ଜୁନ ନିଜର ଜୀବନ ନି...
43,2,4,अर्जुन उवाच\n\nकथं भीष्ममहं संख्ये द्रोणं च मध...,କର୍ମ କରିବା ପାଇଁ ଶ୍ରୀକୃଷ୍ଣଙ୍କ ଆହ୍ୱାନର ପ୍ରତିକ୍ରି...
197,5,1,अर्जुन उवाच\n\nसंन्यासं कर्मणां कृष्ण पुनर्योग...,ଅର୍ଜୁନଙ୍କର ଷୋହଳଟି ପ୍ରଶ୍ନ ମଧ୍ୟରୁ ଏହା ପଞ୍ଚମ ପ୍ରଶ...
219,5,23,शक्नोतीहैव यः सोढुं प्राक्शरीरविमोक्षणात्।\n\n...,ମନୁଷ୍ୟ ଶରୀର ଆତ୍ମାକୁ ତା’ର ଭଗବତ୍ ପ୍ରାପ୍ତି ରୂପୀ ଚ...
674,18,60,स्वभावजेन कौन्तेय निबद्धः स्वेन कर्मणा।कर्तुं ...,ଶ୍ରୀକୃଷ୍ଣ ତାଙ୍କର ଚେତାବନୀ ସୂଚକ ବାଣୀକୁ ଅବ୍ୟାହତ ର...
522,14,7,रजो रागात्मकं विद्धि तृष्णासङ्गसमुद्भवम्।तन्नि...,"ଶ୍ରୀକୃଷ୍ଣ ବର୍ତ୍ତମାନ ରଜୋଗୁଣର କାର୍ଯ୍ୟ, ଏବଂ ଏହା କ..."
201,5,5,यत्सांख्यैः प्राप्यते स्थानं तद्योगैरपि गम्यते...,"ଅଧ୍ୟାତ୍ମିକ ସାଧନାରେ, ବାହ୍ୟ କ୍ରିୟା ଅପେକ୍ଷା, ମନୋଭ..."
239,6,14,प्रशान्तात्मा विगतभीर्ब्रह्मचारिव्रते स्थितः।\...,ଧ୍ୟାନରେ ସଫଳତା ହାସଲ କରିବା ପାଇଁ ଶ୍ରୀକୃଷ୍ଣ ବ୍ରହ୍ମ...


# Mean Pooling

## Cosine Similarity

In [20]:
def mean_cosine(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_mean]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [21]:
verse_index=mean_cosine(query_embedding)

In [22]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
632,18,18,ज्ञानं ज्ञेयं परिज्ञाता त्रिविधा कर्मचोदना।करण...,"କର୍ମ ବିଜ୍ଞାନର ପ୍ରଣାଳୀବଦ୍ଧ ବିଶ୍ଳେଷଣ ଦ୍ୱାରା, ଶ୍ର..."
258,6,33,अर्जुन उवाच\n\nयोऽयं योगस्त्वया प्रोक्तः साम्य...,ଅର୍ଜୁନ ଏହି ଶ୍ଲୋକଟି ଯୋଽୟଂ ଶବ୍ଦରୁ ଆରମ୍ଭ କରିଛନ୍ତି...
71,2,32,यदृच्छया चोपपन्नं स्वर्गद्वारमपावृतम्।\n\nसुखि...,ସମାଜକୁ ସୁରକ୍ଷା ଦେବା ପାଇଁ କ୍ଷତ୍ରିୟମାନଙ୍କର ଆବଶ୍ୟ...
238,6,13,समं कायशिरोग्रीवं धारयन्नचलं स्थिरः।\n\nसंप्रे...,ଧ୍ୟାନରେ ବସିବାର ଆସନ ବିଷୟରେ ବର୍ଣ୍ଣନା କରି ବର୍ତ୍ତମ...
237,6,12,तत्रैकाग्रं मनः कृत्वा यतचित्तेन्द्रियक्रियः।\...,ଧ୍ୟାନରେ ବସିବାର ଆସନ ବିଷୟରେ ବର୍ଣ୍ଣନା କରି ବର୍ତ୍ତମ...
259,6,34,चञ्चलं हि मनः कृष्ण प्रमाथि बलवद्दृढम्।\n\nतस्...,"ଅର୍ଜୁନ, ଆମ ସମସ୍ତଙ୍କୁ ଲକ୍ଷ୍ୟ କରି, ଏହି ଉପଦ୍ରବୀ ମ..."
644,18,30,प्रवृत्तिं च निवृत्तिं च कार्याकार्ये भयाभये।ब...,କୌଣସି ନିର୍ଣ୍ଣୟ ଗ୍ରହଣ କରିବା ପାଇଁ ଆମ୍ଭେମାନେ ସର୍ବ...
72,2,33,अथ चैत्त्वमिमं धर्म्यं संग्रामं न करिष्यसि।\n\...,ଯଦି ଜଣେ ଯୋଦ୍ଧା ଯୁଦ୍ଧକ୍ଷେତ୍ରରେ ଅହିଂସା ଆଚରଣ କରନ୍...
583,16,21,त्रिविधं नरकस्येदं द्वारं नाशनमात्मनः।कामः क्र...,ଶ୍ରୀକୃଷ୍ଣ ବର୍ତ୍ତମାନ ଆସୁରିକ ସ୍ୱଭାବର ଉତ୍ପତ୍ତି ବି...
360,9,30,अपि चेत्सुदुराचारो भजते मामनन्यभाक्।\n\nसाधुरे...,ଭଗବାନଙ୍କ ଭକ୍ତିର ପ୍ରଭାବ ଏତେ ବେଶୀ ଯେ ଏହା ଅତ୍ୟନ୍ତ...


## Euclidean Distance

In [23]:
def mean_ed(query_embedding,counter=10):
    scoring=[ed(query_embedding,i) for i in verse_embeddings_mean]
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [24]:
verse_index=mean_ed(query_embedding)

In [25]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
625,18,11,न हि देहभृता शक्यं त्यक्तुं कर्माण्यशेषतः।यस्त...,ଏଠାରେ ଯୁକ୍ତି କରାଯାଇପାରେ ଯେ କର୍ମଫଳ ତ୍ୟାଗ କରିବା ...
340,9,10,मयाऽध्यक्षेण प्रकृतिः सूयते सचराचरम्।\n\nहेतुन...,"ପୂର୍ବ ଶ୍ଳୋକରେ ବର୍ଣ୍ଣନା କରାଯାଇଥିବା ପ୍ରକାରେ, ଭଗବ..."
439,11,33,तस्मात्त्वमुत्तिष्ठ यशो लभस्व\n\nजित्वा शत्रून...,ଶ୍ରୀକୃଷ୍ଣ ଅର୍ଜୁନଙ୍କ ସମ୍ମୁଖରେ ତାଙ୍କର ଅଭିଳାଷ ପ୍ର...
294,7,22,स तया श्रद्धया युक्तस्तस्याराधनमीहते।\n\nलभते ...,ଲଭତେ ଅର୍ଥ “ସେମାନେ ପ୍ରାପ୍ତ କରନ୍ତି ।” ଦେବତାମାନଙ୍...
142,3,31,ये मे मतमिदं नित्यमनुतिष्ठन्ति मानवाः।\n\nश्रद...,"ଭଗବାନ ଅତି ସୁନ୍ଦର ଭାବରେ, ସେ ବର୍ଣ୍ଣନା କରୁଥିବା ସି..."
604,17,18,सत्कारमानपूजार्थं तपो दम्भेन चैव यत्।क्रियते त...,ଆତ୍ମଶୁଦ୍ଧି ପାଇଁ ତପସ୍ୟା ଏକ ଶକ୍ତିଶାଳୀ ମାଧ୍ୟମ ହୋଇ...
471,12,10,अभ्यासेऽप्यसमर्थोऽसि मत्कर्मपरमो भव।मदर्थमपि क...,ଭଗବାନଙ୍କର ସ୍ମରଣ କରିବା ପାଇଁ ଉପଦେଶ ଦେବା ତାହାକୁ କ...
62,2,23,नैनं छिन्दन्ति शस्त्राणि नैनं दहति पावकः।\n\nन...,"ଚୈତନ୍ୟତା, ଯାହା ଆତ୍ମାର ଲକ୍ଷଣ ଅଟେ, ତାହାର ଅନୁଭବ ମ..."
146,3,35,श्रेयान्स्वधर्मो विगुणः परधर्मात्स्वनुष्ठितात्...,ଏହି ଶ୍ଲୋକରେ ଧର୍ମ ଶବ୍ଦର ବ୍ୟବହାର ଚାରିଥର କରାଯାଇଛି...
63,2,24,अच्छेद्योऽयमदाह्योऽयमक्लेद्योऽशोष्य एव च।\n\nन...,ଆତ୍ମାର ଅମରତ୍ୱ ବିଷୟରେ ଏଠାରେ ପୁନରାବୃତ୍ତି କରାଯାଇଛ...


# Line Matching

## Cosine Similarity

In [26]:
def line_cosine(query_embeddings,counter=10):
    scoring=[]
    for embeddings in verse_embeddings_sentence:
        score=[cosine_similarity(query_embedding,embed) for embed in embeddings]
        scoring.append(max(score))
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [27]:
verse_index=line_cosine(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
45,2,6,न चैतद्विद्मः कतरन्नो गरीयो\n\nयद्वा जयेम यदि ...,ସବୁଠାରୁ ଉପଯୁକ୍ତ କାର୍ଯ୍ୟ ପଦ୍ଧତି ବିଷୟରେ ଅନୁମାନ କ...
207,5,11,कायेन मनसा बुद्ध्या केवलैरिन्द्रियैरपि।\n\nयोग...,ଯୋଗୀମାନେ ଏହା ହୃଦୟଙ୍ଗମ କରିଥାନ୍ତି ଯେ ଆନନ୍ଦ ପ୍ରାପ...
259,6,34,चञ्चलं हि मनः कृष्ण प्रमाथि बलवद्दृढम्।\n\nतस्...,"ଅର୍ଜୁନ, ଆମ ସମସ୍ତଙ୍କୁ ଲକ୍ଷ୍ୟ କରି, ଏହି ଉପଦ୍ରବୀ ମ..."
453,11,47,श्री भगवानुवाच\n\nमया प्रसन्नेन तवार्जुनेदं\n\...,ଅର୍ଜୁନ ଯେହେତୁ ଭୟଭୀତ ହୋଇ ବିଶ୍ୱରୂପକୁ ଲୁକ୍‌କାୟିତ ...
136,3,25,सक्ताः कर्मण्यविद्वांसो यथा कुर्वन्ति भारत।\n\...,"ପୂର୍ବରୁ, ଶ୍ଲୋକ ସଂଖ୍ୟା ୩.୨୦ ରେ, ଶ୍ରୀକୃଷ୍ଣ ଲୋକ-ସ..."
182,4,28,द्रव्ययज्ञास्तपोयज्ञा योगयज्ञास्तथापरे।\n\nस्व...,"ପ୍ରତ୍ୟେକ ମନୁଷ୍ୟ ତାଙ୍କର ସ୍ୱଭାବ, ଇଚ୍ଛା, କର୍ମ, ବୃ..."
197,5,1,अर्जुन उवाच\n\nसंन्यासं कर्मणां कृष्ण पुनर्योग...,ଅର୍ଜୁନଙ୍କର ଷୋହଳଟି ପ୍ରଶ୍ନ ମଧ୍ୟରୁ ଏହା ପଞ୍ଚମ ପ୍ରଶ...
471,12,10,अभ्यासेऽप्यसमर्थोऽसि मत्कर्मपरमो भव।मदर्थमपि क...,ଭଗବାନଙ୍କର ସ୍ମରଣ କରିବା ପାଇଁ ଉପଦେଶ ଦେବା ତାହାକୁ କ...
648,18,34,यया तु धर्मकामार्थान् धृत्या धारयतेऽर्जुन।प्रस...,ଦୃଢ଼ ଇଚ୍ଛାଶକ୍ତି କେବଳ ଯୋଗୀମାନଙ୍କ ଠାରେ ଦେଖାଯାଇ ନ ...
61,2,22,वासांसि जीर्णानि यथा विहाय\n\nनवानि गृह्णाति न...,ଆତ୍ମାର ସ୍ୱରୂପ ସମ୍ବନ୍ଧରେ ଅଧିକ ବର୍ଣ୍ଣନା କରିବାକୁ ...


## Euclidean Distance

In [28]:
def line_ed(query_embeddings,counter=10):
    scoring=[]
    for embeddings in verse_embeddings_sentence:
        score=[ed(query_embedding,embed) for embed in embeddings]
        scoring.append(max(score))
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [29]:
verse_index=line_ed(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
512,13,32,अनादित्वान्निर्गुणत्वात्परमात्मायमव्ययः।शरीरस्...,"ଭଗବାନ ସମସ୍ତ ପ୍ରାଣୀଙ୍କ ହୃଦୟରେ, ପରମାତ୍ମା ରୂପରେ ନ..."
76,2,37,हतो वा प्राप्स्यसि स्वर्गं जित्वा वा भोक्ष्यसे...,୨.୩୧ ଶ୍ଳୋକ ଠାରୁ ଆରମ୍ଭ କରି ଶ୍ରୀକୃଷ୍ଣ ଏ ପର୍ଯ୍ୟନ୍...
364,9,34,मन्मना भव मद्भक्तो मद्याजी मां नमस्कुरु।\n\nमा...,ପୂରା ଅଧ୍ୟାୟରେ ଭକ୍ତି ଏବଂ ଭକ୍ତି ମାର୍ଗ ଉପରେ ଗୁରୁତ...
294,7,22,स तया श्रद्धया युक्तस्तस्याराधनमीहते।\n\nलभते ...,ଲଭତେ ଅର୍ଥ “ସେମାନେ ପ୍ରାପ୍ତ କରନ୍ତି ।” ଦେବତାମାନଙ୍...
455,11,49,मा ते व्यथा मा च विमूढभावो\n\nदृष्ट्वा रूपं घो...,ଶ୍ରୀକୃଷ୍ଣ ଅର୍ଜୁନଙ୍କୁ କ୍ରମାଗତ ଭାବରେ ଆଶ୍ୱାସନା ଦେ...
484,13,4,तत्क्षेत्रं यच्च यादृक् च यद्विकारि यतश्च यत्।...,ବର୍ତ୍ତମାନ ଶ୍ରୀକୃଷ୍ଣ ନିଜେ ଅନେକ ପ୍ରଶ୍ନ ଉପସ୍ଥାପନ ...
481,12,20,ये तु धर्म्यामृतमिदं यथोक्तं पर्युपासते।श्रद्द...,ଅର୍ଜୁନଙ୍କ ପ୍ରଶ୍ନର ଉତ୍ତର ଭାବରେ ଶ୍ରୀକୃଷ୍ଣ ଯାହା ଯ...
83,2,44,भोगैश्वर्यप्रसक्तानां तयापहृतचेतसाम्।\n\nव्यवस...,"ଯେଉଁମାନଙ୍କର ମନ ଇନ୍ଦ୍ରିୟ ସୁଖରେ ଆସକ୍ତ ରହିଥାଏ, ସେ..."
672,18,58,मच्चित्तः सर्वदुर्गाणि मत्प्रसादात्तरिष्यसि।अथ...,"ପୂର୍ବ ଶ୍ଳୋକରେ କ’ଣ କରଣୀୟ ତାହାର ଉପଦେଶ ଦେବା ପରେ, ..."
625,18,11,न हि देहभृता शक्यं त्यक्तुं कर्माण्यशेषतः।यस्त...,ଏଠାରେ ଯୁକ୍ତି କରାଯାଇପାରେ ଯେ କର୍ମଫଳ ତ୍ୟାଗ କରିବା ...


# Whole Part

## Cosine Similarity

In [30]:
def whole_cosine(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_whole]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [31]:
verse_index=whole_cosine(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
258,6,33,अर्जुन उवाच\n\nयोऽयं योगस्त्वया प्रोक्तः साम्य...,ଅର୍ଜୁନ ଏହି ଶ୍ଲୋକଟି ଯୋଽୟଂ ଶବ୍ଦରୁ ଆରମ୍ଭ କରିଛନ୍ତି...
31,1,36,निहत्य धार्तराष्ट्रान्नः का प्रीतिः स्याज्जनार...,"ପୂର୍ବ ଶ୍ଳୋକରେ “ଯଦିଓ” ଶବ୍ଦଟି ଦୁଇଥର ବ୍ୟବହାର କରି,..."
32,1,37,तस्मान्नार्हा वयं हन्तुं धार्तराष्ट्रान्स्वबान...,"ପୂର୍ବ ଶ୍ଳୋକରେ “ଯଦିଓ” ଶବ୍ଦଟି ଦୁଇଥର ବ୍ୟବହାର କରି,..."
364,9,34,मन्मना भव मद्भक्तो मद्याजी मां नमस्कुरु।\n\nमा...,ପୂରା ଅଧ୍ୟାୟରେ ଭକ୍ତି ଏବଂ ଭକ୍ତି ମାର୍ଗ ଉପରେ ଗୁରୁତ...
625,18,11,न हि देहभृता शक्यं त्यक्तुं कर्माण्यशेषतः।यस्त...,ଏଠାରେ ଯୁକ୍ତି କରାଯାଇପାରେ ଯେ କର୍ମଫଳ ତ୍ୟାଗ କରିବା ...
137,3,26,न बुद्धिभेदं जनयेदज्ञानां कर्मसङ्गिनाम्।\n\nजो...,"ମହାପୁରୁଷମାନଙ୍କର ଅଧିକ ଦାୟୀତ୍ୱ ରହିଥାଏ, କାରଣ ସାଧା..."
587,17,1,अर्जुन उवाचये शास्त्रविधिमुत्सृज्य यजन्ते श्रद...,ପୂର୍ବ ଅଧ୍ୟାୟରେ ଶ୍ରୀକୃଷ୍ଣ ଦିବ୍ୟ ଏବଂ ଆସୁରିକ ଗୁଣ ...
330,8,28,वेदेषु यज्ञेषु तपःसु चैव\n\nदानेषु यत्पुण्यफलं...,"ଆମେ ବୈଦିକ ରୀତିରେ କର୍ମକାଣ୍ଡ କରିପାରୁ, ଜ୍ଞାନ ଅର୍ଜ..."
632,18,18,ज्ञानं ज्ञेयं परिज्ञाता त्रिविधा कर्मचोदना।करण...,"କର୍ମ ବିଜ୍ଞାନର ପ୍ରଣାଳୀବଦ୍ଧ ବିଶ୍ଳେଷଣ ଦ୍ୱାରା, ଶ୍ର..."
71,2,32,यदृच्छया चोपपन्नं स्वर्गद्वारमपावृतम्।\n\nसुखि...,ସମାଜକୁ ସୁରକ୍ଷା ଦେବା ପାଇଁ କ୍ଷତ୍ରିୟମାନଙ୍କର ଆବଶ୍ୟ...


## Euclidean Distance

In [32]:
def whole_ed(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_whole]
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [33]:
verse_index=whole_ed(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
412,11,6,पश्यादित्यान्वसून्रुद्रानश्िवनौ मरुतस्तथा।\n\n...,ଭଗବାନଙ୍କର ବିଶ୍ୱରୂପରେ ପୃଥିବୀ ପୃଷ୍ଠର ସମସ୍ତ ଐଶ୍ଚର...
19,1,22,यावदेतान्निरीक्षेऽहं योद्धुकामानवस्थितान्।\n\n...,"ଅର୍ଜୁନ, ସମଗ୍ର ସୃଷ୍ଟିର ପରମ ପ୍ରଭୁ, ଶ୍ରୀକୃଷ୍ଣଙ୍କର..."
18,1,21,अर्जुन उवाच\n\nहृषीकेशं तदा वाक्यमिदमाह महीपते...,"ଅର୍ଜୁନ, ସମଗ୍ର ସୃଷ୍ଟିର ପରମ ପ୍ରଭୁ, ଶ୍ରୀକୃଷ୍ଣଙ୍କର..."
429,11,23,रूपं महत्ते बहुवक्त्रनेत्रं\n\nमहाबाहो बहुबाहू...,"ଭଗବାନଙ୍କର ଅସଂଖ୍ୟ ହସ୍ତ, ପଦ, ମୁଖମଣ୍ଡଳ ଏବଂ ଉଦର ସର..."
449,11,43,पितासि लोकस्य चराचरस्य\n\nत्वमस्य पूज्यश्च गुर...,"ଅର୍ଜୁନ କହୁଛନ୍ତି ଯେ, ଶ୍ରୀକୃଷ୍ଣ ସର୍ବଶ୍ରେଷ୍ଠ ତଥା ..."
323,8,21,अव्यक्तोऽक्षर इत्युक्तस्तमाहुः परमां गतिम्।\n\...,ଆଧ୍ୟାତ୍ମିକ ଜଗତର ଦିବ୍ୟ ଆକାଶକୁ ‘ପରବ୍ୟୋମ’ କୁହାଯାଏ...
689,18,75,व्यासप्रसादाच्छ्रुतवानेतद्गुह्यमहं परम्।योगं य...,"ଶ୍ରୀକୃଷ୍ଣ ଦ୍ୱୈପାୟନ ବ୍ୟାସଦେବ, ଯିଏ ମହର୍ଷି ବେଦବ୍ୟ..."
16,1,19,स घोषो धार्तराष्ट्राणां हृदयानि व्यदारयत्।\n\n...,"ପାଣ୍ଡବ ସେନାର ଶଙ୍ଖନାଦ, କୌରବ ସୈନ୍ୟଙ୍କର ହୃଦୟକୁ ବି..."
30,1,35,एतान्न हन्तुमिच्छामि घ्नतोऽपि मधुसूदन।\n\nअपि ...,ଦ୍ରୋଣାଚାର୍ଯ୍ୟ ଏବଂ କୃପାଚାର୍ଯ୍ୟ ଅର୍ଜୁନଙ୍କ ଗୁରୁ ଥ...
29,1,34,आचार्याः पितरः पुत्रास्तथैव च पितामहाः।\n\nमात...,ଦ୍ରୋଣାଚାର୍ଯ୍ୟ ଏବଂ କୃପାଚାର୍ଯ୍ୟ ଅର୍ଜୁନଙ୍କ ଗୁରୁ ଥ...


# Clustering

## Max Pooling

In [34]:
kmeans_max = KMeans(n_clusters=60, random_state=0).fit(verse_embeddings_max)

In [35]:
def kmeans_max_cluster(query_embedding_list):
    label=kmeans_max.predict(query_embedding_list)[0]
    verse_index=[i for i in range(len(data)) if kmeans_max.labels_[i]==label]
    return verse_index

In [36]:
verse_index=kmeans_max_cluster(query_embedding_list)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
63,2,24,अच्छेद्योऽयमदाह्योऽयमक्लेद्योऽशोष्य एव च।\n\nन...,ଆତ୍ମାର ଅମରତ୍ୱ ବିଷୟରେ ଏଠାରେ ପୁନରାବୃତ୍ତି କରାଯାଇଛ...
121,3,10,सहयज्ञाः प्रजाः सृष्ट्वा पुरोवाच प्रजापतिः।\n\...,ପ୍ରକୃତିର ପ୍ରତ୍ୟେକ ଉପାଦାନ ଭଗବାନଙ୍କ ସୃଷ୍ଟିର ଅପରି...
151,3,40,इन्द्रियाणि मनो बुद्धिरस्याधिष्ठानमुच्यते।\n\n...,କାମନାର ନିବାସ ସ୍ଥାନକୁ ଦର୍ଶାଇ ଶ୍ରୀକୃଷ୍ଣ ବର୍ତ୍ତମା...
233,6,8,ज्ञानविज्ञानतृप्तात्मा कूटस्थो विजितेन्द्रियः।...,"ଜ୍ଞାନର ତାତ୍ପର୍ଯ୍ୟ ଶାବ୍ଦିକ ଜ୍ଞାନ ଅଟେ, ଯାହା ଆମେ ..."
239,6,14,प्रशान्तात्मा विगतभीर्ब्रह्मचारिव्रते स्थितः।\...,ଧ୍ୟାନରେ ସଫଳତା ହାସଲ କରିବା ପାଇଁ ଶ୍ରୀକୃଷ୍ଣ ବ୍ରହ୍ମ...
262,6,37,अर्जुन उवाच\n\nअयतिः श्रद्धयोपेतो योगाच्चलितमा...,ଭଗବତ୍ ପ୍ରାପ୍ତିର ଯାତ୍ରା ଶ୍ରଦ୍ଧାଠାରୁ ଆରମ୍ଭ ହୁଏ ।...
314,8,12,सर्वद्वाराणि संयम्य मनो हृदि निरुध्य च।\n\nमूर...,ଇନ୍ଦ୍ରିୟମାନଙ୍କ ମାଧ୍ୟମରେ ସଂସାର ମନ ମଧ୍ୟକୁ ପ୍ରବେଶ...
330,8,28,वेदेषु यज्ञेषु तपःसु चैव\n\nदानेषु यत्पुण्यफलं...,"ଆମେ ବୈଦିକ ରୀତିରେ କର୍ମକାଣ୍ଡ କରିପାରୁ, ଜ୍ଞାନ ଅର୍ଜ..."
355,9,25,यान्ति देवव्रता देवान् पितृ़न्यान्ति पितृव्रता...,ସାଧକ କେବଳ ତା’ର ଇଷ୍ଟର ସ୍ତର ପର୍ଯ୍ୟନ୍ତ ନିଜକୁ ଉନ୍ନ...
382,10,18,विस्तरेणात्मनो योगं विभूतिं च जनार्दन।\n\nभूयः...,ଆପଣଙ୍କର “... ଅମୃତ ତୁଲ୍ୟ ବାଣୀ ଶୁଣି’’ କହିବା ପରିବ...


## Mean Pooling

In [37]:
kmeans_mean = KMeans(n_clusters=60, random_state=0).fit(verse_embeddings_mean)

In [38]:
def kmeans_mean_cluster(query_embedding_list):
    label=kmeans_mean.predict(query_embedding_list)[0]
    verse_index=[i for i in range(len(data)) if kmeans_mean.labels_[i]==label]
    return verse_index

In [39]:
verse_index=kmeans_mean_cluster(query_embedding_list)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
63,2,24,अच्छेद्योऽयमदाह्योऽयमक्लेद्योऽशोष्य एव च।\n\nन...,ଆତ୍ମାର ଅମରତ୍ୱ ବିଷୟରେ ଏଠାରେ ପୁନରାବୃତ୍ତି କରାଯାଇଛ...
294,7,22,स तया श्रद्धया युक्तस्तस्याराधनमीहते।\n\nलभते ...,ଲଭତେ ଅର୍ଥ “ସେମାନେ ପ୍ରାପ୍ତ କରନ୍ତି ।” ଦେବତାମାନଙ୍...
340,9,10,मयाऽध्यक्षेण प्रकृतिः सूयते सचराचरम्।\n\nहेतुन...,"ପୂର୍ବ ଶ୍ଳୋକରେ ବର୍ଣ୍ଣନା କରାଯାଇଥିବା ପ୍ରକାରେ, ଭଗବ..."
604,17,18,सत्कारमानपूजार्थं तपो दम्भेन चैव यत्।क्रियते त...,ଆତ୍ମଶୁଦ୍ଧି ପାଇଁ ତପସ୍ୟା ଏକ ଶକ୍ତିଶାଳୀ ମାଧ୍ୟମ ହୋଇ...


## Whole

In [40]:
kmeans_whole = KMeans(n_clusters=60, random_state=0).fit(verse_embeddings_whole)

In [41]:
def kmeans_whole_cluster(query_embedding_list):
    label=kmeans_whole.predict(query_embedding_list)[0]
    verse_index=[i for i in range(len(data)) if kmeans_whole.labels_[i]==label]
    return verse_index

In [42]:
verse_index=kmeans_whole_cluster(query_embedding_list)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
116,3,5,न हि कश्िचत्क्षणमपि जातु तिष्ठत्यकर्मकृत्।\n\n...,କେତେକ ବ୍ୟକ୍ତି ଭାବନ୍ତି ଯେ କର୍ମ କେବଳ ବୃତ୍ତିଗତ କର...
146,3,35,श्रेयान्स्वधर्मो विगुणः परधर्मात्स्वनुष्ठितात्...,ଏହି ଶ୍ଲୋକରେ ଧର୍ମ ଶବ୍ଦର ବ୍ୟବହାର ଚାରିଥର କରାଯାଇଛି...
147,3,36,अर्जुन उवाच\n\nअथ केन प्रयुक्तोऽयं पापं चरति प...,ପୁର୍ବ ଶ୍ଲୋକରେ ଶ୍ରୀକୃଷ୍ଣ କହିଲେ ଯେ ଜଣେ ରାଗ ବା ଦ୍...
257,6,32,आत्मौपम्येन सर्वत्र समं पश्यति योऽर्जुन।\n\nसु...,ଆମେ ଶରୀରର ସମସ୍ତ ଅଙ୍ଗ ପ୍ରତ୍ୟଙ୍ଗକୁ ନିଜର ବୋଲି ବିଚ...
625,18,11,न हि देहभृता शक्यं त्यक्तुं कर्माण्यशेषतः।यस्त...,ଏଠାରେ ଯୁକ୍ତି କରାଯାଇପାରେ ଯେ କର୍ମଫଳ ତ୍ୟାଗ କରିବା ...
684,18,70,अध्येष्यते च य इमं धर्म्यं संवादमावयोः।ज्ञानयज...,ଶ୍ରୀକୃଷ୍ଣ ବାରମ୍ବାର ଅର୍ଜୁନଙ୍କୁ ତାଙ୍କର ବୁଦ୍ଧି ଭଗ...
